In [ ]:
import pandas as pd
import scanpy as sc
import anndata as ad
# add local forder to python lib ## why it's not working here but works in command line?
import importlib
from sc_utils import scanpy_utils as su
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
from matplotlib.pyplot import rc_context
import numpy as np
import scipy.sparse as sp
import anndata
from typing import Dict, Optional
from collections import defaultdict
import itertools
import pygame
from matplotlib import cm
import seaborn as sns
from collections import Counter


In [ ]:
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Arial']
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
sns.set_theme(style="ticks", rc=custom_params)
matplotlib.rcParams['figure.figsize'] = [6, 5]


In [ ]:
## check versions
## testing stage, with high verbosity
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.set_figure_params(dpi=900, color_map = 'viridis_r')

In [ ]:
%matplotlib inline

In [ ]:
project_name = 'obese_placenta'
workdir = '/mnt/data/hong/2022/DHJ1_human_obesity_placenta/'
h5dir = f'{workdir}output/cellbender'
os.chdir(workdir)

filter_params = {
    'min_counts':400, 'min_genes':200, 'max_genes' : 5000, 'percent_mt':5, 'percent':3, 'filter_mt':True
}

raw_dict = defaultdict(lambda: "Not Present")
no_doublet_dict = defaultdict(lambda: "Not Present")
filter_dict = defaultdict(lambda: "Not Present")

group = {'Normal_AGA': ['placenta_60', 'placenta_226', 'placenta_248', 'placenta_357', 'placenta_314'], 
         'Obese_AGA': ['placenta_32', 'placenta_81', 'placenta_306', 'placenta_373'], 
         'Obese_LGA': ['placenta_25', 'placenta_40', 'placenta_303', 'placenta_312', 'placenta_330']}
group_inv = defaultdict()
for k, v in group.items():
    for vi in v:
        group_inv[vi] = k
        
for root, sample_list, filenames in os.walk(f'{h5dir}'):
    for sample_name in sample_list:
        if sample_name.startswith('placenta_'):
            raw_dict[sample_name] = su.anndata_from_h5(f'{h5dir}/{sample_name}/{sample_name}_cellbender_filtered.h5')
## given sample order by group
sample_list = list(itertools.chain(*list(group.values())))
for sample_name in sample_list:
    raw_dict[sample_name] = su.anndata_from_h5(f'{h5dir}/{sample_name}/{sample_name}_cellbender_filtered.h5')

In [ ]:
## check hgb and individual specific clusters
ad_all = sc.read_h5ad('output/10x_h5/h5ad/ad_v0.1.h5ad')

In [ ]:
batch_tab = pd.crosstab(ad_all.obs['batch'], [ad_all.obs['group'], ad_all.obs['sex']])

In [ ]:
with pd.ExcelWriter('samples/batch.xlsx') as writer:
    batch_tab.to_excel(writer)

In [ ]:
sum_nuclei = 0
for k,v in raw_dict.items():
    if not k in ['placenta_226', 'placenta_40']:
        sum_nuclei += v.n_obs
sum_nuclei

In [ ]:
doublet_params = {
    'placenta_314':0.09, 'placenta_40':0.09, 'placenta_248' : 0.14, 'placenta_25':0.12, 
    'placenta_226':0.12, 'placenta_60':0.15, 'placenta_373':0.11, 'placenta_32':0.12, 'placenta_303':0.12,
    'placenta_357':0.14, 'placenta_312':0.13, 'placenta_306':0.12, 'placenta_330':0.13, 'placenta_81': 0.14
}
for sample_name, sample in raw_dict.items():
    sample.var_names_make_unique('+')
    sc.external.pp.scrublet(sample, threshold=doublet_params[sample_name])
    # su.doublet_plot(sample_name, sample)

In [ ]:
for sample_name, sample in raw_dict.items():
    doublet = np.array(sample.obs['predicted_doublet'], dtype=bool)
    no_doublet_dict[sample_name] = sample[~doublet]
for sample_name, sample in no_doublet_dict.items():
    su.qc(sample, f'{sample_name}_no_doublet', 'MT', basedir=workdir)
    filter_dict[sample_name] = su.filter_adata(sample, **filter_params)
ad_all = ad.concat(list(filter_dict.values()), label='sample', keys=list(filter_dict.keys()), join='outer', index_unique='-', merge='same')

In [ ]:
sample_list_use = list(itertools.chain(*list(group.values())))
for sample_nouse in ['placenta_226', 'placenta_40']:
    sample_list_use.remove(sample_nouse)

In [ ]:
su.qc(ad_all, f'{project_name}_use_cluster', 'MT', order=sample_list_use, batch_key='sample')

In [ ]:
cell_count_df = ad_all.obs.groupby('sample')['sample'].count()
cell_count_df = cell_count_df.to_frame()
cell_count_df.columns = ['count']


In [ ]:
obs_df = ad_all.obs[['sample', 'group', 'sex']]
## unique rows of obs_df
obs_df = obs_df.drop_duplicates()

In [ ]:
## order cell_count_df_anno by group and sex
obs_df.sort_values(["group", "sex"],
               axis=0, ascending=True,
               inplace=True,
               na_position="first")


In [ ]:
## sc violinplot n_genes by sample
# ad_all.obs['n_genes'] = pd.to_numeric(ad_all.obs['n_genes'] )
sc.set_figure_params(scanpy=True, dpi=80, dpi_save=900, frameon=False, vector_friendly=True, fontsize=14, figsize=(12, 4), color_map=None, format='pdf', facecolor=None, transparent=True, ipython_format='png2x')
sc.pl.violin(ad_all, keys='n_genes', groupby='sample', rotation=90, order = cell_count_df_anno['sample'], stripplot=False, save=f'{project_name}_n_genes.pdf')


In [ ]:
fig = plt.subplots(figsize=(12,4))
bp = sns.barplot(cell_count_df_anno, x='sample', y='count', palette=np.array(['#1f77b4', '#ff7f0e', '#279e68', '#d62728', '#aa40fc', '#8c564b',
       '#e377c2', '#b5bd61', '#17becf', '#aec7e8', '#ffbb78', '#98df8a'])[[9,10,5,7,0,2,8,11,3,6,4,1]], order=cell_count_df_anno['sample'])
bp.set(xlabel=None)
fig = bp.get_figure()
fig.savefig('figures/sample/cell_count_final.pdf')


In [ ]:
ad_all.raw = ad_all
ad_all.layers["raw"] = ad_all.X.copy()
ad_all.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(ad_all, inplace=False)["X"]
)
ad_all.layers["log_norm"] = sc.pp.log1p(sc.pp.normalize_total(ad_all, inplace=False)["X"])

In [ ]:
sc.experimental.pp.recipe_pearson_residuals(ad_all, n_top_genes=2000, batch_key='sample')

In [ ]:
color_dict = {'Normal_AGA' : np.array(pygame.Color('#74b3ce'))/255,
'Obese_AGA' : np.array(pygame.Color('#f7b801'))/255, 
'Obese_LGA' : np.array(pygame.Color('#f7717d'))/255}

mother_color_dict = {'Normal_Weight' : np.array(pygame.Color('#74b3ce'))/255,
'Obesity' : np.array(pygame.Color('#f7717d'))/255}

sex_color_dict = {'Female' : np.array(pygame.Color('#bd6b73'))/255,
'Male' : np.array(pygame.Color('#c6c8ee'))/255}

color_dict = {'Normal_AGA' : np.array(pygame.Color('#74b3ce'))/255,
'Obese_AGA' : np.array(pygame.Color('#f7b801'))/255, 
'Obese_LGA' : np.array(pygame.Color('#f7717d'))/255}

mother_color_dict = {'Normal_Weight' : np.array(pygame.Color('#74b3ce'))/255,
'Obesity' : np.array(pygame.Color('#f7717d'))/255}
trohoblast_cmap = cm.get_cmap('GnBu', 7)
stroma_cmap = cm.get_cmap('PuRd', 7)
immune_cmap = cm.get_cmap('YlOrBr', 8)
type_color_dict = {'VEC' : stroma_cmap(3), 
                   'Fibroblast': stroma_cmap(5), 'Myofibroblast': stroma_cmap(6),
                  'EVT': trohoblast_cmap(2), 'STB': trohoblast_cmap(3),
                  'eSTB': trohoblast_cmap(4), 'CTB': trohoblast_cmap(5),
                  'pCTB': trohoblast_cmap(6), 'Hofbauer cells': immune_cmap(2),
                  'Monocyte': immune_cmap(3), 'B cells': immune_cmap(4),
                  'NK cells': immune_cmap(5), 'T cells': immune_cmap(6)}
# #698f3f

In [ ]:
mother = {'Normal_Weight': ['placenta_60', 'placenta_226', 'placenta_248', 'placenta_357', 'placenta_314'], 
         'Obesity': ['placenta_32', 'placenta_81', 'placenta_306', 'placenta_373', 'placenta_25', 'placenta_40', 'placenta_303', 'placenta_312', 'placenta_330'], }
mother_inv = defaultdict()
for k, v in mother.items():
    for vi in v:
        mother_inv[vi] = k

In [ ]:
sex = {'Male': ['placenta_25', 'placenta_40', 'placenta_81', 'placenta_373', 'placenta_248', 'placenta_314', 'placenta_357'], 
         'Female': ['placenta_303', 'placenta_312', 'placenta_330', 'placenta_32', 'placenta_306', 'placenta_60', 'placenta_226']}
sex_inv = defaultdict()
for k, v in sex.items():
    for vi in v:
        sex_inv[vi] = k

In [ ]:
batch = {'Sept': ['placenta_306', 'placenta_314'], 
         'Oct_Nov': ['placenta_60', 'placenta_226', 'placenta_303', 'placenta_312', 'placenta_32'], 
         'Nov': ['placenta_248', 'placenta_81', 'placenta_25'],
         'Dec': ['placenta_40', 'placenta_330'],
        'June': ['placenta_357', 'placenta_373']}
batch_inv = defaultdict()
for k, v in batch.items():
    for vi in v:
        batch_inv[vi] = k

In [ ]:
ad_all.obs['group'] = ad_all.obs['sample'].map(group_inv)
ad_all.obs['batch'] = ad_all.obs['sample'].map(batch_inv)
ad_all.obs['sex'] = ad_all.obs['sample'].map(sex_inv)
ad_all.obs['mother'] = ad_all.obs['sample'].map(mother_inv)


In [ ]:
hbg = ['HBA1', 'HBA2', 'HBB', 'HBD', 'HBE1', 'HBG1', 'HBG2', 'HBM', 'HBQ1']
sc.tl.score_genes(ad_all, hbg, score_name='Hemoglobins')
cell_cycle_gene = [x.strip() for x in open('/mnt/storage/hong/2021/public_matrices/regev_lab_cell_cycle_genes.txt')]
## how to select the layer to do cell cycle analysis?
su.cell_cycle_analysis(cell_cycle_gene,ad_all,'cell_cycle')


In [ ]:
print(sum(ad_all.obs['Hemoglobins']>1))
pd.crosstab(ad_all.obs['Hemoglobins']>1, ad_all.obs['leiden'])

In [ ]:
tab = pd.crosstab(ad_all.obs['group'], ad_all.obs['leiden'], normalize='columns')
tab.to_csv('data/leiden_group_comp.csv')

In [ ]:
sc.pp.neighbors(ad_all, n_pcs=30)
sc.tl.leiden(ad_all, resolution=1)
sc.tl.umap(ad_all, min_dist=0.2, spread=1)
## show plot inline
# import warnings
# warnings.filterwarnings('ignore')
# %config InlineBackend.figure_formats = ['png']
# sc.pl.umap(ad_all, color='leiden', legend_loc='on data', size=10)
# sc.pl.umap(ad_all, color='Hemoglobins', size=10)
# sc.pl.umap(ad_all, color='phase', size=10)


## raw cluster correlation across samples

In [ ]:
cluster_number_df = pd.crosstab(ad_all[ad_all.obs.Hemoglobins < 0].obs['sample'], ad_all[ad_all.obs.Hemoglobins < 0].obs.leiden, normalize='columns')*100
total_cells = pd.crosstab(ad_all.obs['sample'], ad_all.obs.leiden).sum(axis=0)
# import matplotlib as mpl
# import matplotlib.cm as cm
   
norm = mpl.colors.Normalize(vmin=total_cells.min(), vmax=total_cells.max())
cmap = cm.Blues


m = cm.ScalarMappable(norm=norm, cmap=cmap)
row_colors = total_cells.apply(lambda x: m.to_rgba(x))
# from matplotlib.patches import Patch
plt.figure(figsize=(1, 5))
cbar = plt.colorbar(m)
cbar.set_label('number of cells')
plt.savefig('figures/QC/ncells_cbar_leiden.pdf', bbox_inches = 'tight')
sns.clustermap(cluster_number_df, annot=True, col_colors=row_colors, cbar_pos = (1, 0.5, 0.03, 0.2))
plt.savefig('figures/QC/raw_clusters_byleiden.pdf', bbox_inches = 'tight')

In [ ]:
ad_clean = ad_all[ad_all.obs['leiden']!='26']


In [ ]:

marker_dict_coarse = {
    'Cell cycle': ['ATAD2', 'BRIP1', 'MKI67'],
    'CTB': ['TP63', 'LRP5', 'CDH1'],
    'Syncytin': ['ERVW-1', 'ERVFRD-1'],
    'STB': ['CYP19A1', 'CSH1', 'PSG3'],
    'EVT': ['FSTL3', 'KRT7', 'PRG2'],
    'VEC': ['CD34', 'VWF', 'CDH5'],
    "Artery": ['EFNB2', 'GJA5'],
    #"Vein": ['ACKR1','IL1R1', "SELP", "VCAM1"],
    # "Vein": ['IL1R1'],
    # "Cappilary": ['EMCN'],
    'Fibroblast': ['COL3A1', 'COL1A1', 'DCN'],
    'Actin-Myosin': ['ACTA2', 'TAGLN', 'MYH11'],
    'Leukocytes': ['PTPRC'],
    'Hofbauer cells': ['LYVE1', 'F13A1', 'CD163'],
    'Monocyte': ['ITGAX', 'TYMP', 'SLC11A1'],
    'B cells': ['IGHD', 'MS4A1', 'IGHM'],
    'NK cells': ['GNLY', 'KLRK1', 'KLRD1'],
    'T cells': ['CCR7', 'SELL']
}


In [ ]:
mapping = {
    'Endo-1': 'VEC',
    'Endo-2': 'VEC',
}

In [ ]:
ad_all.obs['subset'].replace(mapping, inplace=True)

In [ ]:
# tropho_intermediate_list = 0:11, 14-16, 20, 22, 24
tropho_intermediate_list = [0,1,2,3,4,5,6,7,9,10,11,14,15,16,20,22,24]
# string all elements in tropho_intermediate_list
tropho_intermediate_list = [str(x) for x in tropho_intermediate_list]

In [ ]:
celltype = {'Trophoblast cell': tropho_intermediate_list,
            'pMSC': ['8', '17'],
            'Endothelial cell': ['13', '19', '21'],
            'Leukocyte': ['12', '18', '23', '25']}
annotate_inv = defaultdict()
for k, v in celltype.items():
    for vi in v:
        annotate_inv[vi] = k
ad_clean.obs['cell type'] = ad_clean.obs['leiden'].map(annotate_inv)


In [ ]:
# sqrt(CPMedian) normalization
ad_clean.layers['sqrtCPMedian'] = ad_clean.layers['raw'].copy()
sc.pp.normalize_total(ad_clean, layer='sqrtCPMedian')
# sqrt transformation
ad_clean.layers['sqrtCPMedian'] = np.sqrt(ad_clean.layers['sqrtCPMedian'])
ad_clean.uns['group_colors'] = np.array(['#74b3ce', '#f7b801', '#f7717d'])


In [ ]:
## write out ad_clean
su.write_adata(ad_all, 'output/10x_h5/h5ad/ad_v0.9_VEC')

In [ ]:
T_NK = ad_clean[ad_clean.obs['leiden']=='18']
sc.pp.neighbors(T_NK, n_pcs=15)
sc.tl.leiden(T_NK, resolution=0.1)
sc.tl.umap(T_NK, min_dist=0.2, spread=1)

sc.pl.umap(T_NK, color='leiden', legend_loc='on data', size=20)


In [ ]:
subset = {'T cells': '2',
            'NK cells': ['0', '1']}
annotate_inv = defaultdict()
for k, v in subset.items():
    for vi in v:
        annotate_inv[vi] = k
T_NK.obs['subset'] = T_NK.obs['leiden'].map(annotate_inv)


In [ ]:
## for grant
# ad_grant = ad_clean[ad_clean.obs.group !='Obese_LGA']
# view = ad_all[np.random.choice(ad_all.n_obs, ad_all.n_obs, replace=False)]
ad_all.obs['subset'] = pd.Categorical(ad_all.obs['subset'], categories=['pCTB', 'CTB', 'eSTB', 'STB', 'EVT',
                                        'VEC', 'Fibroblast', 'Myofibroblast', 'Hofbauer cells', 'Monocyte', 'B cells', 'NK cells', 'T cells'])
new = sc.pp.subsample(ad_all, fraction=1., copy=True)
with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(new, color='subset', add_outline=True, size=20,
               legend_fontsize=12, legend_fontoutline=2, frameon=False,
               title='cell types', palette=type_color_dict, save='group_umap_VEC.pdf')
    
# with rc_context({'figure.figsize': (5, 5)}):
#     sc.pl.umap(new, color='group',
#                legend_fontsize=12, legend_fontoutline=2, frameon=False,
#                title='groups', palette=color_dict, save='group_umap_normaldots.pdf')


In [ ]:
from pandas.api.types import CategoricalDtype
ad_all.obs.subset = ad_all.obs.subset.astype(CategoricalDtype(categories=['pCTB', 'CTB', 'eSTB', 'STB', 'EVT', 'Endo-1', 'Endo-2', 'Fibroblast', 'Myofibroblast', 'Hofbauer cells', 'Monocyte', 'B cells', 'NK cells', 'T cells'],ordered=True))

In [ ]:
# %config InlineBackend.figure_formats = ['png']
# import warnings
# # warnings.filterwarnings('ignore')
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#004369", "white", "#DB1F48"])
sc.pl.dotplot(ad_all, marker_dict_coarse, 'subset', standard_scale='var', layer='log_norm', save='figures/markers_fig1f_artery.pdf')

In [ ]:
subset = {'STB': ['0', '1', '2', '3', '4', '5', '6', '9', '10', '11', '15', '20'],
          'eSTB': ['22'],
          'CTB': ['7', '14'],
          'pCTB': ['16'],
          'Fibroblast': ['8'],
          'Myofibroblast': ['17'],
          'Endo-1': ['13'],
          'Endo-2': ['21'],
          'Endo-3': ['19'],
          'Hofbauer cells': ['12'],
          'T/NK': ['18'],
          'Monocyte': ['23'],
          'B cells': ['25'],
          'EVT': ['24']}
annotate_inv = defaultdict()
for k, v in subset.items():
    for vi in v:
        annotate_inv[vi] = k
ad_all.obs['subset'] = ad_all.obs['leiden'].map(annotate_inv)


In [ ]:
ad_all.write_h5ad('output/10x_h5/h5ad/ad_v0.9_3ENdo.h5ad')

In [ ]:
## replace subset column in ad_clean.obs the column in with T_NK.obs if index exist, otherwise, keep the original value
ad_clean.obs['subset'] = ad_clean.obs['subset'].where(~ad_clean.obs.index.isin(T_NK.obs.index), T_NK.obs['subset'])


In [ ]:
# ad_all.obs['subset'] = pd.Categorical(ad_all.obs['subset'], categories=['pCTB', 'CTB', 'eSTB', 'STB', 'EVT',
#                                         'Endo-1', 'Endo-2', 'Endo-3', 'Fibroblast', 'Myofibroblast', 'Hofbauer cells', 'Monocyte', 'B cells', 'NK cells', 'T cells'])
sc.pl.dotplot(ad_all, marker_dict_coarse, 'subset', standard_scale='var', layer='log_norm', save='figures/markers_fig1f_vectypes_final.pdf')


In [ ]:
# reorder the subset column to ['pCTB', 'CTB', 'eSTB', 'STB', 'EVT', 'Endo-1', 'Endo-2', 'Endo-3', 'Fibroblast',
# 'Myofibroblast', 'Hofbauer cells', 'Monocyte', 'B cells', 'NK cells', 'T cells']
# ad_clean.obs['subset'] = pd.Categorical(ad_clean.obs['subset'], categories=['pCTB', 'CTB', 'eSTB', 'STB', 'EVT', 'Endo-1', 'Endo-2', 'Fibroblast', 'Myofibroblast', 'Hofbauer cells', 'Monocyte', 'B cells', 'NK cells', 'T cells'])
## use palette type_color_dict to color the subset
# sc.pl.umap(ad_clean, color='subset', size=20, palette=type_color_dict)
# sc.pl.umap(ad_clean, color='subset', legend_loc='on data', size=20)
with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(ad_clean, color='subset', add_outline=True, size=20,
               legend_fontsize=12, legend_fontoutline=2, frameon=False,
               title='cell types', palette=type_color_dict, save='celltypes_umap.pdf')


In [ ]:
# ad_clean.X = ad_clean.layers['raw']
# sc.experimental.pp.recipe_pearson_residuals(ad_clean, n_top_genes=2000, batch_key='sample')
ad_clean.obs['Subset'].cat.reorder_categories(['pCTB', 'CTB', 'eSTB', 'STB', 'EVT', 'Endo-1', 'Endo-2', 'Fibroblast', 'Myofibroblast', 'Hofbauer cells', 'Monocyte', '(Memory) B cells', 'NK cells', 'Naive T cells'], inplace=True)

## Automatic annotation

## self annotation

In [ ]:
sc.tl.rank_genes_groups(placenta, 'leiden', method='t-test_overestim_var', key_added='t-test_overestim_var')

In [ ]:
su.find_markers(ad_clean, ['logreg', 't-test', 'wilcoxon', 't-test_overestim_var'], cluster='leiden')

In [ ]:
NK_wilcox_df = pd.DataFrame.from_dict({k: ad_clean.uns['wilcoxon'][k]['NK'] for k in ('names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges')})

In [ ]:
import csv
NK_wilcox_df_sig = NK_wilcox_df.query("pvals_adj <0.05 & logfoldchanges > 1")
NK_wilcox_df_sig.to_csv('output/markers/NK_wilcox_sig.tsv', sep='\t', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
pd.DataFrame(ad_clean.uns['wilcoxon']['names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges'])

In [ ]:
import venn

In [ ]:
#compare cluster1 genes, only stores top 100 by default

wc = pd.DataFrame(ad_clean.uns['t-test']['names']).head(25)
tt = pd.DataFrame(ad_clean.uns['wilcoxon']['names']).head(25)
tt_ov = pd.DataFrame(ad_clean.uns['t-test_overestim_var']['names']).head(25)
logreg = pd.DataFrame(ad_clean.uns['logreg']['names']).head(25)
i = 0
for name in wc.columns:
    plt.figure(i)
    labels = venn.get_labels([set(wc.loc[:,name]),set(tt.loc[:,name]),set(tt_ov.loc[:,name]),set(logreg.loc[:,name])], fill=['number'])
    venn.venn4(labels, names=['Wilcox','T-test','T-test_ov', 'Logreg'] )
    plt.savefig(f'figures/venn_{name}.pdf')
    i += 1

In [ ]:
#sc.settings.set_figure_params(dpi=300)
#visualize the gene expression as an overlay of the umap
#(this way you can visually identify the clusters with a high expression))
for k, v in marker_dict.items():
    sc.pl.umap(ad_all, layer = 'sqrt_norm', color = v, color_map = 'viridis', ncols = 3)

In [ ]:
# Fill in the clusters that belong to each cell type based on each marker in the plot above
cell_dict = {'Trophoblast cells': ['0','1'], 'Smooth muscle cells': ['2'], 
             'Endothelial cells': ['3'],
             'Hofbauer cells': ['4'],
             'T cells': ['5']}

# Initialize empty column in cell metadata
ad_all.obs['annotation'] = np.nan

# Generate new assignments
for i in cell_dict.keys():
    ind = pd.Series(ad_all.obs.leiden).isin(cell_dict[i])
    ad_all.obs.loc[ind,'annotation'] = i

sc.pl.umap(ad_all, color=['annotation'], legend_loc='on data', legend_fontsize=6, save='cell_type_all')

In [ ]:
ad_all.obs['predicted_doublet'] = ad_all.obs['predicted_doublet'].map({True: 'True', False: 'False'})
ad_all.write(filename='output/10x_h5/ad_all.h5ad', compression='gzip')

## composition test
input factors in obs matter

In [ ]:
# # read in samples/sample_2seq.tsv
sample_info = pd.read_csv("samples/samples_2seq.csv", sep=';', decimal=',')
sample_info['sample'] = ['placenta_' + str(sample_id) for sample_id in sample_info['ID']]
sample_info['mother'] = [group.split('_')[0] for group in sample_info.Group]
sample_info['baby'] = [group.split('_')[1] for group in sample_info.Group]
# add Baby Weight from sample_info to ad_clean.obs by key 'sample'
ad_all.obs = ad_all.obs.merge(sample_info, on='sample', how='outer')

In [ ]:
ad_clean.obs['log_ngenes'] = np.log(ad_clean.obs.n_genes)

In [ ]:
## change the colname "Baby Weight_x" to "Birth_weight"
ad_all.obs["group_sex"] = ad_all.obs['group'].astype(
    str) + "_" + ad_all.obs["sex_x"].astype(str)


In [ ]:
import sccoda
import pertpy as pt
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    ad_all,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="subset",
    sample_identifier="sample",
    covariate_obs=["group_sex"],
)

In [ ]:
import sccoda
import pertpy as pt
sccoda_model = pt.tl.Sccoda()
sccoda_ob = sccoda_model.load(
    ad_all,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="subset",
    sample_identifier="sample",
    covariate_obs=["group", "sex"],
)

In [ ]:
viz.stacked_barplot(abund,
                    figsize=(6, 5), 
                    feature_name="group_sex")
plt.savefig('figures/abundance.pdf', dpi=900)

In [ ]:
pt.pl.coda.boxplots(
    sccoda_data,
    modality_key="coda",
    feature_name="group",
    figsize=(12, 5),
    add_dots=True,
    args_swarmplot={"palette": ["red"]},
)

In [ ]:
abund = sccoda_ob.mod["coda"]


In [ ]:
## find a reference cell type
from sccoda.util import data_visualization as viz
viz.rel_abundance_dispersion_plot(
    data=abund,
    abundant_threshold=0.9
)
plt.show()


In [ ]:
# from sccoda.util import comp_ana as mod
# model_all = mod.CompositionalAnalysis(
#     abund, formula="group+sex", reference_cell_type="Endo-2")
# all_results = model_all.sample_hmc(num_results=20000)
all_results.summary_extended(hdi_prob=0.9)


In [ ]:
model_salm_switch_cond = mod.CompositionalAnalysis(abund, formula="C(group, Treatment('Normal_AGA'))", reference_cell_type="eSTB")
switch_results = model_salm_switch_cond.sample_hmc()
switch_results.summary()

In [ ]:
print(switch_results.credible_effects())


In [ ]:
all_results.set_fdr(est_fdr=0.1)
all_results.summary()


In [ ]:
sccoda_model.prepare(
    sccoda_data,
    modality_key="coda",
    reference_cell_type="eSTB",
    formula="group + sex_x"
)


In [ ]:
sim_results = model_salm.sample_hmc()